<a href="https://colab.research.google.com/github/OceaniaCitizen1984/RN-SMOTE-on-Imbalanced-Data/blob/main/src/RN_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing.

## Library Import & Data Extraction

In [161]:
!pip install xgboost

In [162]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

In [163]:
data = pd.read_csv('train_test_network.csv')

## Data Exploration

In [164]:
data.head(10)

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,192.168.1.37,4444,192.168.1.193,49178,tcp,-,290.371539,101568,2592,OTH,...,0,0,-,-,-,-,-,-,1,backdoor
1,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000102,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
2,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000148,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
3,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000113,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
4,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000130,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
5,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.008403,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
6,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000137,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
7,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000143,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
8,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000200,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
9,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000009,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor


In [165]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211043 entries, 0 to 211042
Data columns (total 44 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   src_ip                  211043 non-null  object 
 1   src_port                211043 non-null  int64  
 2   dst_ip                  211043 non-null  object 
 3   dst_port                211043 non-null  int64  
 4   proto                   211043 non-null  object 
 5   service                 211043 non-null  object 
 6   duration                211043 non-null  float64
 7   src_bytes               211043 non-null  int64  
 8   dst_bytes               211043 non-null  int64  
 9   conn_state              211043 non-null  object 
 10  missed_bytes            211043 non-null  int64  
 11  src_pkts                211043 non-null  int64  
 12  src_ip_bytes            211043 non-null  int64  
 13  dst_pkts                211043 non-null  int64  
 14  dst_ip_bytes        

In [166]:
data.drop(columns=['src_ip', 'src_port', 'dst_ip', 'dst_port'], inplace=True)

data.drop(columns=['type'], inplace=True)

In [167]:
data.columns

Index(['proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state',
       'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
       'dns_query', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD',
       'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth',
       'http_method', 'http_uri', 'http_version', 'http_request_body_len',
       'http_response_body_len', 'http_status_code', 'http_user_agent',
       'http_orig_mime_types', 'http_resp_mime_types', 'weird_name',
       'weird_addl', 'weird_notice', 'label'],
      dtype='object')

In [168]:
categorical_columns = [
    'proto',
    'service',
    'conn_state',
    'dns_AA',
    'dns_RD',
    'dns_RA',
    'dns_rejected',
    'dns_query',
    'ssl_version',
    'ssl_cipher',
    'ssl_resumed',
    'ssl_established',
    'ssl_subject',
    'ssl_issuer',
    'http_trans_depth',
    'http_method',
    'http_uri',
    'http_version',
    'http_user_agent',
    'http_orig_mime_types',
    'http_resp_mime_types',
    'weird_name',
    'weird_addl',
    'weird_notice',
]
numerical_columns = set(data.columns) - set(categorical_columns)

In [169]:
data[categorical_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211043 entries, 0 to 211042
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   proto                 211043 non-null  object
 1   service               211043 non-null  object
 2   conn_state            211043 non-null  object
 3   dns_AA                211043 non-null  object
 4   dns_RD                211043 non-null  object
 5   dns_RA                211043 non-null  object
 6   dns_rejected          211043 non-null  object
 7   dns_query             211043 non-null  object
 8   ssl_version           211043 non-null  object
 9   ssl_cipher            211043 non-null  object
 10  ssl_resumed           211043 non-null  object
 11  ssl_established       211043 non-null  object
 12  ssl_subject           211043 non-null  object
 13  ssl_issuer            211043 non-null  object
 14  http_trans_depth      211043 non-null  object
 15  http_method      

In [170]:
total_unique_values = 0
for col in categorical_columns:
    num_unique_values = data[col].nunique()
    print(col, num_unique_values)
    total_unique_values += num_unique_values

print(total_unique_values, "unique categorical values")

proto 3
service 9
conn_state 13
dns_AA 3
dns_RD 3
dns_RA 3
dns_rejected 3
dns_query 726
ssl_version 4
ssl_cipher 6
ssl_resumed 3
ssl_established 3
ssl_subject 6
ssl_issuer 5
http_trans_depth 11
http_method 4
http_uri 86
http_version 2
http_user_agent 35
http_orig_mime_types 3
http_resp_mime_types 10
weird_name 11
weird_addl 3
weird_notice 2
957 unique categorical values


## Data Cleaning

In [171]:
data.drop(columns=['dns_query'], inplace=True)

In [172]:
categorical_columns = list(set(categorical_columns) - set(['dns_query']))
print(categorical_columns)

['ssl_version', 'ssl_subject', 'conn_state', 'http_uri', 'http_resp_mime_types', 'dns_RA', 'service', 'ssl_established', 'weird_notice', 'http_trans_depth', 'ssl_issuer', 'ssl_resumed', 'http_method', 'dns_AA', 'weird_name', 'http_user_agent', 'proto', 'dns_rejected', 'ssl_cipher', 'weird_addl', 'dns_RD', 'http_version', 'http_orig_mime_types']


In [173]:
pd.get_dummies(data, columns=categorical_columns)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_qclass,dns_qtype,...,weird_addl_46,weird_addl_48,dns_RD_-,dns_RD_F,dns_RD_T,http_version_-,http_version_1.1,http_orig_mime_types_-,http_orig_mime_types_application/soap+xml,http_orig_mime_types_application/xml
0,290.371539,101568,2592,0,108,108064,31,3832,0,0,...,False,False,True,False,False,True,False,True,False,False
1,0.000102,0,0,0,1,52,1,40,0,0,...,False,False,True,False,False,True,False,True,False,False
2,0.000148,0,0,0,1,52,1,40,0,0,...,False,False,True,False,False,True,False,True,False,False
3,0.000113,0,0,0,1,48,1,40,0,0,...,False,False,True,False,False,True,False,True,False,False
4,0.000130,0,0,0,1,52,1,40,0,0,...,False,False,True,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211038,65.376610,2665,322,0,5,2925,5,590,0,0,...,False,False,True,False,False,True,False,True,False,False
211039,65.710346,1987,322,0,6,2307,5,590,0,0,...,False,False,True,False,False,True,False,True,False,False
211040,65.766512,3922,322,0,7,4294,6,642,0,0,...,False,False,True,False,False,True,False,True,False,False
211041,65.753940,2401,322,0,6,2721,5,590,0,0,...,False,False,True,False,False,True,False,True,False,False


In [174]:
data[list(numerical_columns)]

,http_request_body_len,http_status_code,dns_qtype,dns_qclass,duration,dst_bytes,dst_pkts,src_ip_bytes,src_bytes,dst_ip_bytes,missed_bytes,dns_rcode,label,http_response_body_len,src_pkts
0,0,0,0,0,290.371539,2592,31,108064,101568,3832,0,0,1,0,108
1,0,0,0,0,0.000102,0,1,52,0,40,0,0,1,0,1
2,0,0,0,0,0.000148,0,1,52,0,40,0,0,1,0,1
3,0,0,0,0,0.000113,0,1,48,0,40,0,0,1,0,1
4,0,0,0,0,0.000130,0,1,52,0,40,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211038,0,0,0,0,65.376610,322,5,2925,2665,590,0,0,1,0,5
211039,0,0,0,0,65.710346,322,5,2307,1987,590,0,0,1,0,6
211040,0,0,0,0,65.766512,322,6,4294,3922,642,0,0,1,0,7
211041,0,0,0,0,65.753940,322,5,2721,2401,590,0,0,1,0,6


In [175]:
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

data_n = data

data_n.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,290.371539,101568,2592,OTH,0,108,108064,31,...,0,0,0,-,-,-,-,-,-,1
1,tcp,-,0.000102,0,0,REJ,0,1,52,1,...,0,0,0,-,-,-,-,-,-,1
2,tcp,-,0.000148,0,0,REJ,0,1,52,1,...,0,0,0,-,-,-,-,-,-,1
3,tcp,-,0.000113,0,0,REJ,0,1,48,1,...,0,0,0,-,-,-,-,-,-,1
4,tcp,-,0.000130,0,0,REJ,0,1,52,1,...,0,0,0,-,-,-,-,-,-,1


In [176]:
data_n[list(numerical_columns)] = pd.DataFrame(standard_scaler.fit_transform(data[list(numerical_columns)]))
data_n[list(numerical_columns)] = pd.DataFrame(robust_scaler.fit_transform(data[list(numerical_columns)]))
data_n[list(numerical_columns)] = pd.DataFrame(min_max_scaler.fit_transform(data[list(numerical_columns)]))

In [177]:
data_n.head()


,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,3.105016e-03,0.000026,6.622629e-07,OTH,0.0,0.004386,0.016568,0.000254,...,0.0,0.0,0.0,-,-,-,-,-,-,1.0
1,tcp,-,1.090712e-09,0.000000,0.000000e+00,REJ,0.0,0.000041,0.000008,0.000008,...,0.0,0.0,0.0,-,-,-,-,-,-,1.0
2,tcp,-,1.582601e-09,0.000000,0.000000e+00,REJ,0.0,0.000041,0.000008,0.000008,...,0.0,0.0,0.0,-,-,-,-,-,-,1.0
3,tcp,-,1.208337e-09,0.000000,0.000000e+00,REJ,0.0,0.000041,0.000007,0.000008,...,0.0,0.0,0.0,-,-,-,-,-,-,1.0
4,tcp,-,1.390123e-09,0.000000,0.000000e+00,REJ,0.0,0.000041,0.000008,0.000008,...,0.0,0.0,0.0,-,-,-,-,-,-,1.0


In [178]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from time import time
from sklearn.model_selection import train_test_split

# try 85/15 and 90/10
# make a graph to show accuracies

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data_n['label'])  # Replace 'label' with the correct target column name

# cats_partial = {
#                  # 'ssl_issuer', 'http_orig_mime_types', 'ssl_cipher', 'ssl_resumed', 'http_uri',
#                  'http_user_agent',
#                 'proto',
#                 'http_resp_mime_types',
#                 'dns_rejected',
#                 'http_method',
#                  'weird_name', 'dns_RD', 'dns_AA', 'ssl_subject', 'service', 'weird_notice', 'http_version',
#                  'dns_RA', 'weird_addl', 'ssl_version',
#     #             'http_trans_depth',
#     # 'ssl_established',
#     'conn_state',
# }
cats_partial = {
                 'ssl_issuer', 'http_orig_mime_types', 'ssl_cipher', 'ssl_resumed', 'http_uri',
                 'http_user_agent',
                'proto',
                'http_resp_mime_types',
                'dns_rejected',
                'http_method',
                 'weird_name', 'dns_RD', 'dns_AA', 'ssl_subject', 'service', 'weird_notice', 'http_version',
                 'dns_RA', 'weird_addl', 'ssl_version',
                'http_trans_depth',
    'ssl_established',
    'conn_state',
}

# df_n = pd.get_dummies(df, columns=categorical_columns)
x = data_n[list(cats_partial.union(numerical_columns).difference({'label'}))]
x = pd.get_dummies(x, columns=list(cats_partial))

# Clean column names to avoid special characters
x.columns = x.columns.str.replace(r"[^A-Za-z0-9_]", "", regex=True)


# Model with Different

## We will be try different balancing techinques:


*   All Features
*   SMOTE
*   RN-SMOTE

## We will use XGBoost to test all of them.



## Imbalanced data

In [179]:
Imbalanced_data = x

In [180]:
X_train, X_test, y_train, y_test = train_test_split(Imbalanced_data, y, test_size=0.1, random_state=2)

In [181]:
y_train = y_train.values.ravel() if isinstance(y_train, pd.DataFrame) else y_train
X_train = X_train.values if not isinstance(X_train, (pd.DataFrame, np.ndarray)) else X_train


In [185]:
# Ensure column names are valid
X_train.columns = X_train.columns.str.replace(r"[^A-Za-z0-9_]", "", regex=True)

# Ensure all columns are numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')

# Convert X_train to NumPy if needed
X_train = X_train.values

# Train the model
model = XGBClassifier()
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = model.predict(X_test)

In [184]:
print(type(X_train), X_train.shape)
print(type(y_train), y_train.shape)

<class 'pandas.core.frame.DataFrame'> (189938, 245)
<class 'numpy.ndarray'> (189938,)
